In [ ]:
%pip install crewai_tools crewai -U

In [ ]:
%pip install --upgrade crewai crewai_tools

In [ ]:
pip show crewai

In [ ]:
%pip install litellm

## main

In [ ]:
# Initialize all your components
from crewai import Crew, Process, LLM
#import agentops
from dotenv import load_dotenv
import json 
#from datetime import datetime
# Import your other agents from their respective files
from RFP_Strategic_Scout_A1 import selected_agent
from Contextual_Analyst_A2 import Extractor_agent
from llm_as_judge_A3 import judge_agent
import os 
load_dotenv()

#LLM

groq_llm = LLM(
    model="groq/llama-3.1-8b-instant",
    temperature=0.1
)
groq_llm2 = LLM(
    model="groq/llama-3.3-70b-versatile",
    temperature=0.1
)

openai_llm = LLM(
    model="openai/gpt-4o-mini", 
    temperature=0.1
)


#main.py
def main():

    CURRENT_DIR = os.getcwd() 
    BASE_DIR = os.path.dirname(CURRENT_DIR) 
    md_path = os.path.join(BASE_DIR, "data", "processed", "RFP_Final_Cleaned.md")
    output_dir = os.path.join(BASE_DIR, "outputs")
    #Agents
    selected= selected_agent(llm=openai_llm,md_path=md_path,output_dir=output_dir)
    Extractor= Extractor_agent(llm=openai_llm,md_path=md_path,output_dir=output_dir)
    judge=judge_agent(llm=openai_llm,output_dir=output_dir)
    Extractor.set_context_dependency(selected.get_task)
    judge.set_context_dependency(Extractor.get_task)
    

    # Create the crew
    crew = Crew(
        agents=[
            selected.get_agent,
            Extractor.get_agent,
            judge.get_agent
    
        ],
        tasks=[
            selected.get_task,
            Extractor.get_task,
            judge.get_task
            
        ],
        process=Process.sequential,
        max_rpm=2,
        
     
    )
    
   
    result = crew.kickoff()
    print("=== RESULTS ===")
    print(result)
    return result

if __name__ == "__main__":
    main()

test def

In [ ]:
import re
import json
from pathlib import Path

# 1. تعديل الدالة لتكون مستقلة للاختبار (حذف self)


def cut_full_text(md_path: Path, titles: list):
    """
    Extracts the full text content of specific sections from the RFP markdown file.
    """
    # التأكد من أن المسار موجود
    if not Path(md_path).exists():
        return f"Error: MD file not found at {md_path}"
        
    with open(md_path, 'r', encoding='utf-8') as f:
        content = f.read()
        
    extracted = []
    for title in titles:
        clean_title = title.replace('#', '').strip()

        # باترن مرن للبحث عن العناوين
        pattern = rf"(?ms)^#*\s*.*{re.escape(clean_title)}.*?\n(.*?)(?=\n#|\n\d+\s*-\s|\Z)"
        match = re.search(pattern, content)
        if match: 
            text_content = match.group(1).strip()
            extracted.append(f"SECTION: {title}\nCONTENT:\n{text_content}")
        else:
            extracted.append(f"SECTION: {title}\nCONTENT: [Text not found]")

    return "\n\n" + "="*30 + "\n\n".join(extracted)



# --- 2. إعداد المسارات ---
mdpath = r"C:\Users\user\OneDrive - University of Prince Mugrin\سطح المكتب\Capstone_Project_SDAIA\src\data\processed\RFP_Final_Cleaned.md"
json_path = r"C:\Users\user\OneDrive - University of Prince Mugrin\سطح المكتب\Capstone_Project_SDAIA\src\outputs\selected_sections.json"

# --- 3. استخراج العناوين من ملف الـ JSON أولاً ---
try:
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        # استخراج مصفوفة العناوين (تأكدي من مطابقة مفتاح JSON، هنا افترضت أنه 'selected_sections')
        # إذا كان الـ JSON عبارة عن قائمة مباشرة استخدمي: titles_list = [item['section_name'] for item in data]
        titles_list = [item['section_name'] for item in data.get('selected_sections', [])]
except Exception as e:
    print(f"Error loading JSON: {e}")
    titles_list = []

# --- 4. استدعاء الدالة بالبيانات الصحيحة ---
if titles_list:
    result = cut_full_text(mdpath, titles_list)
    print("=== المخرجات المستخرجة ===")
    print(result)
else:
    print("قائمة العناوين فارغة، تأكدي من ملف JSON")

In [ ]:
mdpath=r"C:\Users\user\OneDrive - University of Prince Mugrin\سطح المكتب\Capstone_Project_SDAIA\src\data\processed\RFP_cleaned.md"
title=r"C:\Users\user\OneDrive - University of Prince Mugrin\سطح المكتب\Capstone_Project_SDAIA\src\outputs\Filter_sections.json"
a=cut_full_text(mdpath,title)

In [ ]:
import re
import json

def get_contextual_toc(md_path):
    with open(md_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # تقسيم النص بناءً على العناوين (Markdown # أو الأرقام مثل 1 -)
    # هذا النمط يلتقط العناوين التقليدية والرقمية في الكراسات السعودية
    sections = re.split(r'\n(?=#+\s|\d+\s*-\s)', content)
    
    smart_toc = []
    
    for section in sections:
        lines = section.strip().split('\n')
        if not lines:
            continue
            
        title = lines[0].strip()
        # دمج الأسطر التالية للعنوان لتكوين المعاينة (تجاهل الأسطر الفارغة)
        body_preview = " ".join([l.strip() for l in lines[1:] if l.strip()])[:1000] 
        
        smart_toc.append({
            "title": title,
            "preview": body_preview + "..." if body_preview else "لا يوجد نص متاح"
        })
        
    return smart_toc

In [ ]:
path=r"C:\Users\user\OneDrive - University of Prince Mugrin\سطح المكتب\Capstone_Project_SDAIA\src\Agents\request1_final_clean3.md"
result=get_contextual_toc(path)

In [ ]:
result

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# تحميل المتغيرات من ملف .env
load_dotenv()

# استدعاء المفتاح
api_key = os.getenv("OPENAI_API_KEY")

# تمرير المفتاح للعميل
client = OpenAI(api_key=api_key)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Hello"}]
)

print(response.choices[0].message.content)

In [ ]:
import socket
try:
    host = socket.gethostbyname("api.groq.com")
    print(f"Connection successful: {host}")
except Exception as e:
    print(f"Connection failed: {e}")

In [ ]:
import re

def get_sections_content(md_file_path, selected_headings):
    with open(md_file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    extracted_data = {}
    
    # تقسيم الملف بناءً على العناوين (Headers) التي تبدأ بـ # أو ## أو ###
    # سنبحث عن العناوين المختارة ونستخرج ما تحتها
    for heading in selected_headings:
        # نمط Regex للبحث عن العنوان وما بعده حتى العنوان التالي
        # نستخدم re.escape للتعامل مع أي رموز خاصة في اسم العنوان
        escaped_heading = re.escape(heading)
        pattern = rf"(^#+.*{escaped_heading}.*$\n)([\s\S]*?)(?=\n#+|$)"
        
        match = re.search(pattern, content, re.MULTILINE | re.IGNORECASE)
        if match:
            extracted_data[heading] = match.group(2).strip()
        else:
            extracted_data[heading] = "لم يتم العثور على محتوى لهذا القسم."

    return extracted_data

In [ ]:
selected_titles = [" الاستبعاد من المنافسة"," الموافقة على الشروط"]
filtered_context = get_sections_content(r"C:\Users\user\OneDrive - University of Prince Mugrin\سطح المكتب\Capstone_Project_SDAIA\src\Agents\request1_final_clean3.md", selected_titles)

In [ ]:
filtered_context

In [ ]:
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    raise ValueError("لم يتم العثور على GOOGLE_API_KEY. تأكد من إضافته لملف .env")


In [ ]:
%pip install "unstructured[md]"

read


In [ ]:
import os
from agentic_doc.parse import parse

# 1. تحديد المسارات
pdf_path = r"C:\Users\user\OneDrive - University of Prince Mugrin\سطح المكتب\Capstone_Project_SDAIA\src\data\request1.pdf"
output_folder = r"C:\Users\user\OneDrive - University of Prince Mugrin\سطح المكتب\Capstone_Project_SDAIA\src\data\processed"

# إنشاء المجلد إذا لم يكن موجوداً
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 2. عملية التحويل
print("جاري تحويل الملف من PDF إلى MD...")
results = parse([pdf_path],extract_images=False)

# 3. حفظ النتيجة كملف .md
# نفترض أن results عبارة عن قائمة كائنات، نأخذ النص من العنصر الأول
md_filename = "request1_raw.md"
md_output_path = os.path.join(output_folder, md_filename)

with open(md_output_path, "w", encoding="utf-8") as f:
    # ملاحظة: تأكد من مخرجات مكتبة agentic_doc، عادة النص يكون في .text أو .markdown
    f.write(results[0].markdown) 

print(f"✅ تم حفظ الملف الخام في: {md_output_path}")



clean 

In [ ]:
import re
from unstructured.partition.md import partition_md
from unstructured.cleaners.core import clean_extra_whitespace

def auto_clean_with_unstructured(file_path):
    # 1. تقسيم الملف إلى عناصر
    elements = partition_md(filename=file_path)

    cleaned_text = []

    for element in elements:
        text = element.text.strip()
        
        # --- الفلترة بناءً على بداية النص (أوصاف الصور واللوقو) ---
        image_keywords = [
            "Summary :", "logo:", "Visible Elements :", "Placement & Dimensions :", 
            "Analysis :", "Graphic Elements :", "Text Elements :", "Design & Placement :", 
            "Design & Layout :", "Layout :", "Design Elements :", "Design Details :", 
            "Text Fields :", "Colour Palette :", "Spatial Relationships :","Dimensions & Placement :","/25, 4:29 PM","تاريخ طرح الكراسة: /1446","/1446"
        ]
        
        if any(text.startswith(key) for key in image_keywords):
            continue 

        # --- الفلترة بناءً على وجود كلمات معينة (الترويسات والتذييلات) ---
        excluded_headers = [
            "المملكة العربية السعودية", "كراسة الشروط", "تاريخ الإصدار", 
            "tenders.etimad.sa", "المعتمد بموجب قرار وزير المالية","اسم المنافسة: ","رقم الكراسة: ","تاريخ طرح الكراسة:" ,"/25, 4:29 PM"
        ]
        if any(key in text for key in excluded_headers):
            continue

        # --- تصحيح حذف أرقام الصفحات والروابط باستخدام Regex ---
        # حذف أرقام الصفحات مثل 31/37
        text = re.sub(r'\d+/\d+', '', text)
        
        # حذف أي رابط يبدأ بـ http (روابط منصة اعتماد)
        text = re.sub(r'https?://\S+', '', text)

        # 4. تنظيف الفراغات الناتجة عن الحذف
        text = clean_extra_whitespace(text)

        # إضافة النص النظيف فقط إذا لم يصبح فارغاً بعد الحذف
        if text.strip():
            cleaned_text.append(text)

    return "\n\n".join(cleaned_text)

# --- التشغيل ---
input_file = r"C:\Users\user\OneDrive - University of Prince Mugrin\سطح المكتب\Capstone_Project_SDAIA\src\data\processed\request1_raw.md"
final_content = auto_clean_with_unstructured(input_file)

with open("request1_final_clean3.md", "w", encoding="utf-8") as f:
    f.write(final_content)

print("✅ تم تنظيف الملف بنجاح وحذف أوصاف الصور والروابط وأرقام الصفحات!")

In [ ]:

def read_toc_only(file_path, limit_lines=145):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = [next(f) for _ in range(limit_lines)]
    return "".join(lines)

# ثم نمرر هذا النص للـ Task مباشرة
toc_content = read_toc_only(r"C:\Users\user\OneDrive - University of Prince Mugrin\سطح المكتب\Capstone_Project_SDAIA\src\Agents\request1_final_clean3.md")

In [ ]:
print(toc_content)

In [ ]:
import json
import os

def read_toc_to_json(file_path, output_json_path, limit_lines=145):
    lines = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            # نقرأ الأسطر ونقوم بتنظيفها من الفراغات الزائدة
            for _ in range(limit_lines):
                line = next(f).strip()
                if line: # نتجاهل الأسطر الفارغة تماماً لتقليل حجم الملف
                    lines.append(line)
    except StopIteration:
        pass # انتهى الملف قبل الوصول للحد المحدد
    except Exception as e:
        print(f"حدث خطأ أثناء القراءة: {e}")
        return None

    # تشكيل هيكل البيانات للـ JSON
    toc_data = {
        "document_name": os.path.basename(file_path),
        "toc_lines": lines  # هنا نضع القائمة
    }

    # حفظ الملف بصيغة JSON
    with open(output_json_path, 'w', encoding='utf-8') as json_file:
        json.dump(toc_data, json_file, ensure_ascii=False, indent=4)
    
    print(f"✅ تم استخراج الفهرس وحفظه في: {output_json_path}")
    return toc_data



In [ ]:
import re
from unstructured.partition.md import partition_md
from unstructured.cleaners.core import clean_extra_whitespace

def auto_clean_with_unstructured(file_path):
    elements = partition_md(filename=file_path)
    cleaned_text = []

    for element in elements:
        text = element.text.strip()
        
        # --- الفلترة الكلمات المستبعدة ---
        image_keywords = [
            "Summary :", "logo:", "Visible Elements :", "Placement & Dimensions :", 
            "Analysis :", "Graphic Elements :", "Text Elements :", "Design & Placement :", 
            "Design & Layout :", "Layout :", "Design Elements :", "Design Details :", 
            "Text Fields :", "Colour Palette :", "Spatial Relationships :","Dimensions & Placement :","/25, 4:29 PM","تاريخ طرح الكراسة: /1446","/1446"
        ]        
        if any(text.startswith(key) for key in image_keywords): continue 

        excluded_headers = [
            "المملكة العربية السعودية", "كراسة الشروط", "تاريخ الإصدار", 
            "tenders.etimad.sa", "المعتمد بموجب قرار وزير المالية","اسم المنافسة: ","رقم الكراسة: ","تاريخ طرح الكراسة:" ,"/25, 4:29 PM"
        ]        
        if any(key in text for key in excluded_headers): continue

        # --- معالجة النص المنظف ---
        text = re.sub(r'\d+/\d+', '', text)
        text = re.sub(r'https?://\S+', '', text)
        text = clean_extra_whitespace(text)

        if text.strip():
            # --- ميزة استعادة العناوين (The Fix) ---
            # إذا كان السطر يبدأ بنمط عنوان (رقم - نص) ولم يكن مصنفاً كعنوان، نحوله يدوياً
            # النمط: رقم متبوع بمسافة ثم شرطة ثم مسافة (مثلاً: 5 - أهلية)
            heading_pattern = r'^(\d+\s*-\s+.*|^القسم\s+.*|^المادة\s+.*)'
            
            if re.match(heading_pattern, text):
                # نتأكد أنه ليس ضمن قائمة العناوين المحولة مسبقاً لمنع التكرار
                formatted_text = f"## {text}" 
                cleaned_text.append(formatted_text)
            else:
                cleaned_text.append(text)

    return "\n\n".join(cleaned_text)


# --- التشغيل ---
input_file = r"C:\Users\user\OneDrive - University of Prince Mugrin\سطح المكتب\Capstone_Project_SDAIA\src\data\processed\request1_raw.md"
final_content = auto_clean_with_unstructured(input_file)

with open("request1_final_clean3.md", "w", encoding="utf-8") as f:
    f.write(final_content)

print("✅ تم تنظيف الملف بنجاح وحذف أوصاف الصور والروابط وأرقام الصفحات!")

## propsal_convertion

In [ ]:
import os
import re
import json
from agentic_doc.parse import parse
from unstructured.partition.md import partition_md
from unstructured.cleaners.core import clean_extra_whitespace

class DocumentProcessor:
    def __init__(self, output_folder):
        self.output_folder = output_folder
        if not os.path.exists(self.output_folder):
            os.makedirs(self.output_folder)

    def process_pdf_to_clean_md(self, pdf_path):
        """
        المهمة: تحويل PDF إلى MD ثم تنظيفه وإصلاح العناوين في خطوة واحدة.
        """
        file_base_name = os.path.basename(pdf_path).replace(".pdf", "")
        raw_md_path = os.path.join(self.output_folder, f"{file_base_name}_raw.md")
        final_md_path = os.path.join(self.output_folder, f"{file_base_name}_final_clean.md")

        # 1. المرحلة الأولى: القراءة والتحويل (Parsing)
        print(f"🔄 جاري تحويل {file_base_name} من PDF إلى Markdown...")
        results = parse([pdf_path])
        raw_content = results[0].markdown

        with open(raw_md_path, "w", encoding="utf-8") as f:
            f.write(raw_content)

        # 2. المرحلة الثانية: التنظيف الهيكلي (Cleaning & Repairing)
        print(f"🧹 جاري تنظيف النص وإصلاح العناوين (بما في ذلك عنوان 64)...")
        cleaned_content = self._auto_clean_logic(raw_md_path)

        with open(final_md_path, "w", encoding="utf-8") as f:
            f.write(cleaned_content)

        print(f"✅ اكتملت المعالجة! الملف النظيف: {final_md_path}")
        return final_md_path

    def _auto_clean_logic(self, file_path):
        """منطق التنظيف العميق واستعادة العناوين"""
        elements = partition_md(filename=file_path)
        cleaned_text = []

        # كلمات مستبعدة (الصور والترويسات)
        image_keywords = [
            "Summary :", "logo:", "Visual Elements  :", "Placement & Dimensions :", 
            "Analysis :", "Graphic Elements :", "Text Elements :", "Design & Placement :", 
            "Design & Layout :", "Layout :", "Design Elements :", "Design Details :", 
            "Text Fields :", "Colour Palette :", "Spatial Relationships :","Dimensions & Placement :","/25, 4:29 PM","تاريخ طرح الكراسة: /1446","/1446","Technical Details :","Design & Colour :","Logo Design :","Text Details :"
        ]   
        excluded_headers = [
            "المملكة العربية السعودية", "كراسة الشروط", "تاريخ الإصدار", 
            "tenders.etimad.sa", "المعتمد بموجب قرار وزير المالية","اسم المنافسة: ","رقم الكراسة: ","تاريخ طرح الكراسة:" ,"/25, 4:29 PM"
        ]    

        for element in elements:
            text = element.text.strip()
            
            # الفلترة بناءً على الكلمات المستبعدة
            if any(text.startswith(key) for key in image_keywords): continue 
            if any(key in text for key in excluded_headers): continue

            # تنظيف الروابط وأرقام الصفحات
            text = re.sub(r'\d+/\d+', '', text)
            text = re.sub(r'https?://\S+', '', text)
            text = clean_extra_whitespace(text)

            if text.strip():
                # --- تحسين التعرف على العناوين (Fixing Heading 64 and others) ---
                # النمط الجديد: يبحث عن رقم في بداية السطر يتبعه (شرطة أو نقطة أو مسافة)
                # مثال: "64 عنوان" أو "64 - عنوان" أو "64. عنوان"
                heading_pattern = r'^(\d+[\s\.\-].*|^القسم\s+.*|^المادة\s+.*)'
                
                if re.match(heading_pattern, text):
                    # إذا لم يكن السطر يحمل علامة العنوان #، نضيفها له
                    if not text.startswith('#'):
                        text = f"## {text}"
                
                cleaned_text.append(text)

        return "\n\n".join(cleaned_text)

# --- مثال على طريقة الاستخدام في ملفك الرئيسي ---
if __name__ == "__main__":
    PDF_INPUT = r"C:\Users\user\OneDrive - University of Prince Mugrin\سطح المكتب\Capstone_Project_SDAIA\src\data\RFP.pdf"
    OUTPUT_DIR = r"C:\Users\user\OneDrive - University of Prince Mugrin\سطح المكتب\Capstone_Project_SDAIA\src\data\processed"
    
    processor = DocumentProcessor(OUTPUT_DIR)
    final_file = processor.process_pdf_to_clean_md(PDF_INPUT)

In [ ]:
import os
import glob
from proposal_processor import run_pipeline  # Ensure your file is named proposal_processor.py

# --- CONFIGURATION ---
RAW_DIR = r"D:\Capstone_Project_SDAIA\src\data\raw"
PROCESSED_DIR = r"D:\Capstone_Project_SDAIA\src\data\processed"

def process_all_vendors():
    # 1. Create output directory if it doesn't exist
    if not os.path.exists(PROCESSED_DIR):
        os.makedirs(PROCESSED_DIR)
        print(f"📁 Created directory: {PROCESSED_DIR}")

    # 2. Find all files starting with "Vendor" (case-insensitive search)
    # This captures .pdf and .docx files
    search_pattern = os.path.join(RAW_DIR, "Vendor*.*")
    vendor_files = glob.glob(search_pattern)

    if not vendor_files:
        print(f"⚠️ No files starting with 'Vendor' found in {RAW_DIR}")
        return

    print(f"🔍 Found {len(vendor_files)} vendor files to process.")

    # 3. Loop through each file and run your chunked cleaning pipeline
    for file_path in vendor_files:
        try:
            print(f"\n--- Processing: {os.path.basename(file_path)} ---")
            
            # This triggers your gpt-4o-mini chunked refinement
            final_path = run_pipeline(file_path, PROCESSED_DIR)
            
            print(f"✅ Success! Cleaned Markdown saved at: {final_path}")
        
        except Exception as e:
            # Catches errors like the 30k token limit or API issues
            print(f"❌ Failed to process {os.path.basename(file_path)}: {str(e)}")

if __name__ == "__main__":
    process_all_vendors()

In [ ]:
import os
import glob
from crewai import Crew, Process
from langchain_openai import ChatOpenAI
from Response_analyst_A3 import ResponseAnalyst # Ensure this matches your file name

# --- CONFIGURATION ---
# The input is the output of your proposal_processor
INPUT_DIR = r"D:\Capstone_Project_SDAIA\src\data\processed"
# The requirements come from Agent 3 (LLM as Judge)
REQUIREMENTS_JSON = r"D:\Capstone_Project_SDAIA\src\outputs\strategic_refined_requirements.json"
# Final evidence location
OUTPUT_DIR = r"D:\Capstone_Project_SDAIA\src\outputs"

def run_batch_analysis():
    # 1. Initialize the LLM (Using gpt-4o-mini to stay under 30k token limit)
    llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

    # 2. Find all files starting with "Vendor" and ending with "cleaned.md"
    search_pattern = os.path.join(INPUT_DIR, "Vendor*cleaned.md")
    cleaned_files = glob.glob(search_pattern)

    if not cleaned_files:
        print(f"⚠️ No cleaned vendor files found in {INPUT_DIR}")
        return

    print(f"🔍 Found {len(cleaned_files)} cleaned proposals to analyze.")

    # 3. Iterate through each vendor file
    for md_file in cleaned_files:
        vendor_filename = os.path.basename(md_file)
        print(f"\n🚀 Starting RAG Extraction for: {vendor_filename}")

        try:
            # Initialize your Agent 4 class
            # It uses the Judge's output as the source of truth for requirements
            analyst_system = ResponseAnalyst(
                llm=llm,
                proposal_md_path=md_file,
                requirements_json_path=REQUIREMENTS_JSON,
                output_dir=OUTPUT_DIR
            )

            # Wrap in a Crew for execution
            crew = Crew(
                agents=[analyst_system.agent],
                tasks=[analyst_system.task],
                process=Process.sequential,
                verbose=True
            )

            # Kickoff the RAG search
            result = crew.kickoff()
            
            # Validation
            evidence_path = os.path.join(OUTPUT_DIR, f"evidence_{vendor_filename}.json")
            if os.path.exists(evidence_path):
                print(f"✅ Success! Evidence extracted to: {evidence_path}")
            
        except Exception as e:
            print(f"❌ Error analyzing {vendor_filename}: {str(e)}")

if __name__ == "__main__":
    run_batch_analysis()